# II. Data explore

## Import các thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

## Load dataset đã tạo ở phần trước

In [4]:
raw_df = pd.read_csv("../Data/raw/raw_data.csv")

In [ ]:
raw_df.head()

,Company,Year,EPS,Earnings,Revenue,Marketcap,Total Debts,Net Assets,Total Assets,Cash on hand,Country,Share price,Categories
0,Apple,2023,$6.16,$114.30 B,$383.28 B,$3.043 T,$111.08 B,$62.14 B,$352.58 B,$61.55 B,USA,$195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
1,Apple,2022,$5.92,$113.96 B,$387.53 B,$2.066 T,$120.06 B,$50.67 B,$352.75 B,$48.30 B,USA,$195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
2,Apple,2021,$6.08,$116.90 B,$378.32 B,$2.901 T,$124.71 B,$63.09 B,$351.00 B,$62.63 B,USA,$195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
3,Apple,2020,$3.74,$74.25 B,$294.13 B,$2.255 T,$112.43 B,$65.33 B,$323.88 B,$90.94 B,USA,$195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
4,Apple,2019,$3.19,$66.15 B,$267.68 B,$1.287 T,$108.04 B,$90.48 B,$338.51 B,$100.55 B,USA,$195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."


## Số dòng và số cột

In [5]:
n_rows, n_cols = raw_df.shape
print(f"Dataset có {n_rows} dòng và {n_cols} cột")

Dataset có 69460 dòng và 13 cột


## Kiểm tra sự trùng lắp giữa các dòng

In [6]:
n_dups = raw_df.duplicated().sum()

if n_dups:
    print(f"Có {n_dups} dòng trùng nhau")
else:
    print("Không có dòng bị trùng")

Có 23506 dòng trùng nhau


Xóa các dòng bị trùng lắp

In [7]:
raw_df.drop_duplicates(inplace=True)
if raw_df.duplicated().sum():
    print(f"Có {raw_df.duplicated().sum()} dòng trùng nhau")
else:
    print("Không có dòng bị trùng")

Không có dòng bị trùng


In [8]:
raw_df.sample(5)

,Company,Year,EPS,Earnings,Revenue,Marketcap,Total Debts,Net Assets,Total Assets,Cash on hand,Country,Share price,Categories
42056,Straumann,2011,$0.91,$0.17 B,$0.78 B,$2.66 B,NaN,$0.71 B,$0.86 B,$0.40 B,Switzerland,$136.90,Medical devices
48900,Almarai,2021,$0.42,$0.44 B,$4.22 B,$12.92 B,$2.72 B,$4.43 B,$8.46 B,$0.16 B,Saudi Arabia,$15.24,🍴 Food
39737,Tesco,2007,$0.59,$5.21 B,$83.74 B,NaN,$11.19 B,$20.76 B,$48.72 B,$2.04 B,UK,$3.68,"🛍️ Retail, 🛒 Supermarket Chains"
48094,Dassault Aviation,2022,$4.48,$0.84 B,$8.20 B,$14.02 B,$0.25 B,$6.43 B,$24.13 B,$10.30 B,France,$196.85,"✈️ Aircraft manufacturers, 🔫 Defense contracto..."
1997,Philip Morris,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,$91.31,🚬 Tobacco


## Ý nghĩa của các cột

- `Company`: tên công ty

- `Year`: năm ghi nhận

- `EPS`: lợi nhuận / cổ phần

- `Earnings`: lợi nhuận

- `Revenue`: doanh thu

- `Marketcap`: vốn hoá thị trường

- `Total Debts`: tổng nợ

- `Net Assets`: tài sản ròng

- `Total Assets`: tổng tài sản

- `Cash on hand`: tiền mặt có sẵn

- `Country`: quốc giá

- `Share price`: giá cổ phiếu

- `Categories`: danh mục công ty

## Chuẩn hoá dữ liệu

EPS, Earnings, Revenue, Marketcap, Total Debts, Net Assets, Total Assets, Cash on hand, Share price: Các cột này chứa dữ liệu số nhưng có kèm theo ký hiệu đơn vị như "$", "B" (tượng trưng cho tỷ đô la). Do đó, ta cần loại bỏ các ký tự không cần thiết và chuyển đổi sang kiểu float.  
    Trong cột, có các kí hiệu `T`, `B`, `M` tượng trưng cho ` nghìn tỷ `, `  tỷ ` ,  ` triệu ` . Ta sẽ chuyển hết về `B`.

In [10]:
# df = raw_df.copy()

cols_to_convert = ['EPS', 'Earnings', 'Revenue', 'Marketcap', 'Total Debts', 'Net Assets', 'Total Assets', 'Cash on hand', 'Share price']

conversion_factors = {'M': 1e-3, 'T': 1e3}

for col in cols_to_convert:
    raw_df[col] = raw_df[col].replace({'\$': '', ',': ''}, regex=True)
    last_char = raw_df[col].str[-1]

    for idx, c in last_char.items():
        if c in ['M', 'T']:
            raw_df.at[idx, col] = pd.to_numeric(raw_df.at[idx, col][:-1], errors='coerce') * conversion_factors.get(c)
        elif c == 'B':
            raw_df.at[idx, col] = pd.to_numeric(raw_df.at[idx, col][:-1], errors='coerce')
        else:
            raw_df.at[idx, col] = pd.to_numeric(raw_df.at[idx, col], errors='coerce')


raw_df.sample(5)

<ipython-input-10-354830f9d4b2>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  raw_df[col] = raw_df[col].replace({'\$': '', ',': ''}, regex=True)


AttributeError: Can only use .str accessor with string values!

In [11]:
raw_df[raw_df['Company']== 'Apple']

,Company,Year,EPS,Earnings,Revenue,Marketcap,Total Debts,Net Assets,Total Assets,Cash on hand,Country,Share price,Categories
0,Apple,2023,6.160000,114.3,383.28,3043.0,111.08,62.14,352.58,61.55,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
1,Apple,2022,5.920000,113.96,387.53,2066.0,120.06,50.67,352.75,48.3,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
2,Apple,2021,6.080000,116.9,378.32,2901.0,124.71,63.09,351.0,62.63,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
3,Apple,2020,3.740000,74.25,294.13,2255.0,112.43,65.33,323.88,90.94,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
4,Apple,2019,3.190000,66.15,267.68,1287.0,108.04,90.48,338.51,100.55,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
5,Apple,2018,3.070000,67.97,261.61,746.07,114.48,107.14,365.72,66.3,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
6,Apple,2017,2.450000,64.25,239.17,860.88,115.68,134.04,375.31,74.18,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
7,Apple,2016,2.100000,59.21,218.11,608.96,87.03,128.24,321.68,67.15,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
8,Apple,2015,2.370000,71.15,234.98,583.61,64.46,119.35,290.47,41.6,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."
9,Apple,2014,1.870000,59.28,199.8,643.12,35.29,111.54,231.83,25.07,USA,195.71,"👨‍💻 Software, 👩‍💻 Tech, 🔌 Electronics, 🇺🇸 Dow ..."


Ta cũng cần thay đổi lại tên các cột `EPS`, `Earnings`, `Revenue`, `Marketcap`, `Total Debts`, `Net Assets`, `Total Assets`, `Cash on hand`, `Share price` để dễ thấy ý nghĩa của cột.

In [12]:
new_column_names = {
    'EPS': 'EPS ($)',
    'Earnings': 'Earnings ($B)',
    'Revenue': 'Revenue ($B)',
    'Marketcap': 'Marketcap ($B)',
    'Total Debts': 'Total Debts ($B)',
    'Net Assets': 'Net Assets ($B)',
    'Total Assets': 'Total Assets ($B)',
    'Cash on hand': 'Cash on hand ($B)',
    'Share price': 'Share price ($)'
}

raw_df = raw_df.rename(columns=new_column_names)

Bây giờ, cột `Categories` chứa các icon không cần thiết và gây nhiều khó khăn cho nên ta sẽ xoá các icon này

Hàm loại bỏ emoji

In [13]:
def remove_emojis(input_string):
    # Create a regex pattern to match a broader range of special characters
    special_char_pattern = re.compile("[^\w\s,]\s?", flags=re.UNICODE)

    # Use re.sub to replace the matched characters with an empty string
    result_string = special_char_pattern.sub('', input_string)

    return result_string

Xoá các icon trong `Categories`

In [14]:
raw_df["Categories"] = raw_df["Categories"].apply(lambda x: x if pd.isna(x) else remove_emojis(x))

In [15]:
raw_df["Categories"][22:24]

,Categories
22,"Software, Tech, Electronics, Dow jones, Tech H..."
23,"Software, Tech, Video games, Dow jones, AI"


Đã xoá thành công emoji, tuy nhiên vẫn còn các khoảng trắng dư thừa

Viết hàm chuẩn hoá để xoá bỏ các khoảng trắng dư thừa

In [16]:
def normalize(x): # x is a row
    if pd.isna(x):
        return x
    else:
        x = x.strip()
        x = x.split(", ")
        x = [cate.strip() for cate in  x]
        x = ", ".join(x)
        return x



raw_df["Categories"] = raw_df["Categories"].apply(normalize)

## Kiểm tra missing-values của các cột

In [17]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45954 entries, 0 to 69459
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Company            45954 non-null  object 
 1   Year               45954 non-null  int64  
 2   EPS ($)            32237 non-null  float64
 3   Earnings ($B)      32371 non-null  object 
 4   Revenue ($B)       32503 non-null  object 
 5   Marketcap ($B)     34364 non-null  object 
 6   Total Debts ($B)   34199 non-null  object 
 7   Net Assets ($B)    36745 non-null  object 
 8   Total Assets ($B)  36824 non-null  object 
 9   Cash on hand ($B)  36043 non-null  object 
 10  Country            45954 non-null  object 
 11  Share price ($)    45954 non-null  object 
 12  Categories         42642 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 5.9+ MB


Tính tỉ lệ missing-values của các cột

In [18]:
raw_df.apply(lambda col: 1 -col.isna().sum() / len(col))

,0
Company,1.000000
Year,1.000000
EPS ($),0.701506
Earnings ($B),0.704422
Revenue ($B),0.707294
Marketcap ($B),0.747791
Total Debts ($B),0.744201
Net Assets ($B),0.799604
Total Assets ($B),0.801323
Cash on hand ($B),0.784328


- Nhìn chung, có nhiều cột có missing-values khá nhiều

- Đối với các observation thiếu đi `Categories` thì ta sẽ drop. Lí do: công ty không có thông tin về danh mục thì khó cho việc phân tích.

- Đối với các cột còn lại (đa số là `Numerical`), ta sẽ thay bằng giá trị `median` tương ứng dựa theo công ty đó.

## Xử lý missing-values cho các cột `Categorical`

Tiến hành bỏ các dòng missing-values ở `Categories`

In [19]:
raw_df.dropna(subset=["Categories"], inplace=True)


Kiểm tra lại số lượng missing-values ở cột `Categories`

In [20]:
print(f'Số lượng missing-values: {raw_df["Categories"].isna().sum()}')

Số lượng missing-values: 0


Xem lại shape

In [21]:
raw_df.shape

(42642, 13)

## Xử lý missing values cho các cột `Numerical`

In [22]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42642 entries, 0 to 69459
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Company            42642 non-null  object 
 1   Year               42642 non-null  int64  
 2   EPS ($)            29863 non-null  float64
 3   Earnings ($B)      30014 non-null  object 
 4   Revenue ($B)       30136 non-null  object 
 5   Marketcap ($B)     31765 non-null  object 
 6   Total Debts ($B)   31705 non-null  object 
 7   Net Assets ($B)    34028 non-null  object 
 8   Total Assets ($B)  34105 non-null  object 
 9   Cash on hand ($B)  33375 non-null  object 
 10  Country            42642 non-null  object 
 11  Share price ($)    42642 non-null  object 
 12  Categories         42642 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 4.6+ MB


Trong các cột `Numerical`, cột tồn tại missing values bao gồm:

- `EPS ($)`           

- `Earnings ($B)`       

- `Revenue ($B)`      

- `Marketcap ($B)`     

- `Total Debts ($B)`   

- `Net Assets ($B)`    

- `Total Assets ($B)`  

- `Cash on hand ($B)`


In [23]:
num_missing_cols = ["EPS ($)", "Earnings ($B)", "Revenue ($B)", "Marketcap ($B)", "Total Debts ($B)", "Net Assets ($B)", "Total Assets ($B)", "Cash on hand ($B)"]
num_missing_cols

['EPS ($)',
 'Earnings ($B)',
 'Revenue ($B)',
 'Marketcap ($B)',
 'Total Debts ($B)',
 'Net Assets ($B)',
 'Total Assets ($B)',
 'Cash on hand ($B)']

Phương án xử lý missing-value bằng cách replace bằng `median` hay `mean` theo cột có vẻ không hợp lý bởi vì có nhiều công ty và quy mô cũng khác nhau. Do đó nếu thay như vậy thì giảm mức độ chính xác đi rất nhiều.

Một phương án có thể đó là thay bằng giá trị `median` dựa trên từng công ty.

Tuy nhiên, có trường hợp một `company` có một cột toàn `nan`. Do đó, ta sẽ drop luôn những `company` thuộc trường hợp này.

### Tạo DataFrame mới lưu số lượng missing-values của mỗi cột theo từng company

In [25]:
missing_counts_df = raw_df.groupby("Company")[num_missing_cols].apply(lambda company: company.isna().sum())

# Chuyển `Company` thành cột
missing_counts_df = missing_counts_df.reset_index()
missing_counts_df

,Company,EPS ($),Earnings ($B),Revenue ($B),Marketcap ($B),Total Debts ($B),Net Assets ($B),Total Assets ($B),Cash on hand ($B)
0,(HLBank) Hong Leong Bank,11,11,11,5,5,4,4,5
1,37 Interactive Entertainment,9,9,9,19,10,7,7,7
2,3M,0,0,0,0,1,1,1,1
3,3i Group,4,3,3,6,4,3,3,5
4,7-Eleven,9,9,9,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...
1849,monday.com,19,19,19,20,19,19,19,19
1850,nVent Electric,16,16,16,17,18,16,16,16
1851,Ørsted,14,14,14,16,13,13,13,13
1852,ČEZ Group,4,10,10,5,5,5,5,5


### Tạo DataFrame chứa số lượng missing values là lớn nhất trong các cột theo công ty

In [26]:
max_missing = missing_counts_df.apply(lambda row: max(row[num_missing_cols]), axis = 1)
max_missing_df = pd.concat((missing_counts_df["Company"], max_missing), axis = 1)
max_missing_df.columns = ["Company", "Max missing values"]
max_missing_df

,Company,Max missing values
0,(HLBank) Hong Leong Bank,11
1,37 Interactive Entertainment,19
2,3M,1
3,3i Group,6
4,7-Eleven,9
...,...,...
1849,monday.com,20
1850,nVent Electric,18
1851,Ørsted,16
1852,ČEZ Group,10


Do mỗi công ty, ta thu thập `23` observation tương ứng với `23` năm nên ta sẽ drop những observation nào có `Max mising values` là `23`. Bởi vì các company này tồn tại các cột không thể tính `median`

Lấy danh sách công ty cần drop

In [27]:
drop_list = max_missing_df[max_missing_df["Max missing values"] == 23]["Company"]
drop_list.head()

,Company
8,ADNOC Drilling Company
9,ADNOC Gas
10,ADNOC Logistics & Services
37,Abu Dhabi Commercial Bank (ADCB)
38,Abu Dhabi Islamic Bank (ADIB)


In [28]:
print(f"Số lượng công ty cần drop: {len(drop_list)}")

Số lượng công ty cần drop: 52


Tiến hành drop các công ty này trong `raw_df`

In [29]:
raw_df = raw_df[~raw_df["Company"].isin(drop_list.values)]

Xem lại shape

In [30]:
raw_df.shape

(41446, 13)

Bây giờ, ta có thể tiến hành replace `nan` bằng `median` của cột theo từng công ty mà không sợ bị lỗi nữa.

Tương ứng với từng `Numerical` cột, ta sẽ tạo một dictionary với:

- Key: tên của Company

- Value: `median` tương ứng với cột dựa theo Company

In [31]:
median_dict = []

for col in num_missing_cols:
    median_dict.append(raw_df.groupby("Company")[col].median().to_dict())


Lặp qua các cột `Numerical` có missing value và tiến hành thay thế các giá trị `nan` bằng `median` tương ứng

In [32]:
for i, col in enumerate(num_missing_cols):
    raw_df[col] = raw_df.apply(lambda row: median_dict[i][row["Company"]] if pd.isna(row[col]) else row[col], axis = 1)

In [33]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41446 entries, 0 to 69459
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Company            41446 non-null  object 
 1   Year               41446 non-null  int64  
 2   EPS ($)            41446 non-null  float64
 3   Earnings ($B)      41446 non-null  float64
 4   Revenue ($B)       41446 non-null  float64
 5   Marketcap ($B)     41446 non-null  float64
 6   Total Debts ($B)   41446 non-null  float64
 7   Net Assets ($B)    41446 non-null  float64
 8   Total Assets ($B)  41446 non-null  float64
 9   Cash on hand ($B)  41446 non-null  float64
 10  Country            41446 non-null  object 
 11  Share price ($)    41446 non-null  object 
 12  Categories         41446 non-null  object 
dtypes: float64(8), int64(1), object(4)
memory usage: 4.4+ MB


## Lưu preprocessed data

In [35]:
raw_df.to_csv("../Data/preprocessed/cleaned_dataset.csv", index=False)